In [1]:
import pandas as pd
import os
import pickle
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk.corpus.reader.util import read_line_block
from nltk.corpus import stopwords
from tqdm import tqdm


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>


In [2]:
# load data
filename = '2018.csv'
wd = os.getcwd()
data_dir = os.path.join(wd, 'data')
raw_data = pd.read_csv(os.path.join(data_dir, filename), header=None, skiprows=1)

In [3]:
# preprocess data
filename_pp = 'preprocessed.csv'
data = raw_data[raw_data.iloc[:, 0] != '[deleted]']
data = data[data.iloc[:, 0] != '[removed]']
data.to_csv(os.path.join(data_dir, filename_pp), header=False, index=False)

In [4]:
# create corpus
newcorpus = PlaintextCorpusReader(data_dir, fileids=filename_pp, para_block_reader=read_line_block)

In [5]:
# preprocess corpus
stemmer=nltk.PorterStemmer()
lemmatizer=nltk.WordNetLemmatizer()
sentences = []

for sent in tqdm(newcorpus.sents()):
    sent_processed = []
    
    for w in sent:
        w = w.lower()
        if not w.isalpha():
            continue # remove punctuation
        if w in stopwords.words('english'):
            continue # remove stopwords
        if stemmer:
            w = stemmer.stem(w)
        if lemmatizer:
            w = lemmatizer.lemmatize(w)
        sent_processed.append(w)
        
    if sent_processed:
        sentences.append(sent_processed)

100%|██████████| 1178813/1178813 [58:22<00:00, 336.56it/s] 


In [6]:
with open(os.path.join(data_dir,'sentences_dump.pkl'), 'wb') as f:
    pickle.dump(sentences, f)